# Loan Approval Prediction
Edmund Walsh - May 10th, 2020

## Introduction
The project examines the data provided by the Home Mortgage Disclosure Act (HMDA) which requires mortgage lenders in the United States to disclose information about the mortgage lending decisions they have made. Specifically, we will be examining prediction of whether or not an application will be accepted or denied.

This notebook is part of a short 5-day project whose purpose goes beyond the acceptance prediction of mortgages. The focus of this project is more about the process and end-to-end engineering from raw data to results and presentation. This notebook will focus on the data science approach and process and highlight a roadmap best illustrated in the image below:

<img src='https://miro.medium.com/max/1400/1*LoBdYL_YyIcYJ842peLDpQ.jpeg'
     alt='Data Science & Data Engineering'
     style='height: 350px; width: 750px;' />

## A Ground Up Approach
The pyramid above is such a good illustration because the process truly is building in a step-by-step fashion 
towards the ultimate goal of finding useful results that are actionable and impactful in the real world. 
The end results get all of the attention, but a project is unlikely to be successful without these strong foundations.

## Context 
While this project request didn't specifically state 'why' we are looking into this data, I will work within the context of three important rationales.
1. Mortgage due diligence is expensive and time intensive. A process that can more reliably expidite the process will save lenders significant time and resources.
2. From a regulatory perspective and also importantly as Machine Intelligence becomse a larger and more common part of this process it is important for us to be aware of and highlight any bias.
3. Some financial instituions may be more or less likely to issue mortgages and this may reflect either an over or under utilization of the balance sheet or their risk appetite.

### Preparation
Before digging in, let's install our python requirements and follow the instructions for setting up our docker environment and tools in the [README.md](file:///../README.md)

In [1]:
pip install --user -r ./misc/requirements.txt

Note: you may need to restart the kernel to use updated packages.


### Package Import
Now we can begin importing the required packages. Many of these are common python packages, the exception is seed which is a set of functions we will use to import our initial data and begin our collect step on our roadmap.

In [2]:
import seed
import pandas as pd
import numpy as np
import os
import config
import math
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

### A Discussion of the data
Our data sources will come from three major sources and we will use their APIs to download the data.
1. Our main dataset comes from the HMDA database and includes not only a large set of information about the individual loan approval decision but also information about the originating institutions.
    a. A full list of available data on the mortgage approvals can be found [here](http://cfpb.github.io/api/hmda/fields.html)
    b. We also will look at the originating instituions and information about that dataset can be found [here](https://api.consumerfinance.gov/data/hmda/slice/institutions/metadata)
2. Our next and complementary set of information comes from the census bureau. We will use their county business patterns series which aggregate economic information at a county level. I hope that this information will provide some valueable economic insight into regional economics that may affect an mortgage approval decision.
    a. Details about this dataset can be found [here](https://www.census.gov/programs-surveys/cbp.html)
    b. This API requires a key which you can sign up for [here](https://api.census.gov/data/key_signup.html)
    c. After you have signed up, please include this key in the [config.py](file:///./config.py) file.
3. Finally, we will also fill in some data from Federal Reserve Bank of St. Louis. This data will come into use towards the end of the project as we begin to look across time periods as it should give us an indication of both the financial conditions and sentiment of the originating institutions.
    a. Details about this API can be found [here](https://fred.stlouisfed.org/docs/api/fred/)

In [3]:
# after configuring the census API re-import config and check api key
import config
print(config.api_key)

8dadaedad2b940dd8ffff397507286b479540d00


### Data Collection
Luckily for us, the designers of the APIs have made this pretty easy. A big thank you to them!

To start, I have selected a single year and a single state. Feel free to change to your preferences, data is available from 2007 - 2017. Some important caveats. As this is an illustrative project only, there are some important details about availability of data (i.e. when it was published) and data type issues that would require more attention in a production environment.

In [4]:
# choose a first state by two letter code and year
init_state = "OH"
init_yr = 2016

In [5]:
# pull data from the HMDA database on mortgage approvals -- this may take awhile
data_lar = seed.lar_pull(init_state, init_yr)

In [6]:
# a quick data snapshot
data_lar.head() 

,action_taken,action_taken_name,agency_code,agency_abbr,agency_name,applicant_ethnicity,applicant_ethnicity_name,applicant_income_000s,applicant_race_1,applicant_race_2,...,state_name,hud_median_family_income,loan_amount_000s,number_of_1_to_4_family_units,number_of_owner_occupied_units,minority_population,population,rate_spread,tract_to_msamd_income,uuid
0,1,Loan originated,7,HUD,Department of Housing and Urban Development,2,Not Hispanic or Latino,75,5,NaN,...,Ohio,66600,219,3165,2746,9.869999885559082,10439,NaN,148.33999633789062,b73debb0-35a0-4a26-b982-aed4973e497c
1,3,Application denied by financial institution,7,HUD,Department of Housing and Urban Development,2,Not Hispanic or Latino,60,5,NaN,...,Ohio,69100,293,3131,2921,16.459999084472656,8742,NaN,172.69000244140625,601c85f7-eafe-4dec-9908-541fea2016ed
2,1,Loan originated,7,HUD,Department of Housing and Urban Development,2,Not Hispanic or Latino,87,5,NaN,...,Ohio,66600,104,2320,1485,4.429999828338623,5508,NaN,89.05000305175781,73761754-e4e8-4170-a1b7-1136bba44d1a
3,1,Loan originated,7,HUD,Department of Housing and Urban Development,2,Not Hispanic or Latino,86,5,NaN,...,Ohio,55400,153,948,809,2.490000009536743,2973,NaN,133.8300018310547,bc299795-2c39-4051-b90a-00a4f7e18695
4,1,Loan originated,7,HUD,Department of Housing and Urban Development,2,Not Hispanic or Latino,46,5,NaN,...,Ohio,66600,206,2375,2210,5.340000152587891,6044,NaN,126.06999969482422,581d5ed1-773f-42fa-91f4-502e1554cff8


In [7]:
'{:,.0f}'.format(data_lar.shape[0]) + ' total rows  for a total of ' + \
'{:,.0f}'.format(data_lar.shape[0]*data_lar.shape[1]) + ' data points'

'493,271 total rows  for a total of 38,968,409 data points'

In [8]:
# Now pull county business patters data from the census bureau
census_df = seed.census_pull(init_state, init_yr, data_lar, config.api_key)

In [9]:
# A quick snapshot
census_df.head()

,EMP,ESTAB,PAYANN,POP,county_code,county_name,state_abbr,state_code,year
0,3899081,195687,210202807,1250871,35,"Cuyahoga County, Ohio",OH,39,2016
1,3775265,169990,190400631,1138190,49,"Franklin County, Ohio",OH,39,2016
2,466581,35316,20664563,227255,85,"Lake County, Ohio",OH,39,2016
3,201710,14425,7926456,111289,169,"Wayne County, Ohio",OH,39,2016
4,2809075,125607,159909829,782863,61,"Hamilton County, Ohio",OH,39,2016


In [10]:
'{:,.0f}'.format(census_df.shape[0]) + ' total rows  for a total of ' + \
'{:,.0f}'.format(census_df.shape[0]*census_df.shape[1]) + ' data points'

'88 total rows  for a total of 792 data points'

In [11]:
# Finally, let's pull data about the originating institutions
data_inst = seed.inst_pull(init_state, init_yr)

In [12]:
# A quick snapshot
data_inst.head()

,activity_year,respondent_id,agency_code,agency_abbr,agency_name,federal_tax_id,respondent_name,respondent_address,respondent_city,respondent_state,...,parent_state,parent_zip_code,respondent_name_panel,respondent_city_panel,respondent_state_panel,other_lender_code,region_code,validity_error,assets,lar_count
0,2016,0000000046,1,OCC,Office of the Comptroller of the Currency,31-4247738,FIRST NATIONAL BANK OF MCCONNE,"86 N. KENNEBEC AVENUE, PO BOX 208",MCCONNELSVILLE,OH,...,OH,43756,FIRST NB,MCCONNELSVILLE,OH,0,3,N,138285,129
1,2016,0000000047,1,OCC,Office of the Comptroller of the Currency,35-0704860,FIRST FINANCIAL BANK NA,1401 S 3RD ST,TERRE HAUTE,IN,...,IN,47807,FIRST FNCL BK NA,TERRE HAUTE,IN,0,3,N,2882577,2695
2,2016,0000000086,1,OCC,Office of the Comptroller of the Currency,31-0294798,FIRST NATIONAL BANK OF GERMANT,17 NORTH MAIN STREET,GERMANTOWN,OH,...,NaN,NaN,FIRST NB,GERMANTOWN,OH,0,3,N,52364,38
3,2016,0000000324,1,OCC,Office of the Comptroller of the Currency,23-0916895,FIRST NATIONAL BANK AND TRUST,40 SOUTH STATE ST,NEWTOWN,PA,...,NaN,NaN,FIRST NB&TC NEWTOWN,NEWTOWN,PA,0,1,N,860869,184
4,2016,0000000325,1,OCC,Office of the Comptroller of the Currency,24-0558097,"FNB BANK, NA",354 MILL STREET,DANVILLE,PA,...,PA,17604,FNB BK NA,DANVILLE,PA,0,1,N,363285,270


In [13]:
'{:,.0f}'.format(data_inst.shape[0]) + ' total rows  for a total of ' + \
'{:,.0f}'.format(data_inst.shape[0]*data_inst.shape[1]) + ' data points'

'100 total rows  for a total of 2,400 data points'

### Data Storage
We are very early on, but we have already pulled together a rather large dataset. So, after building on our foundataion of data collection, where we know have functions that can pull and update the necessary data, we need to start thinking about data storage and the tools we are going to need to scale up analysis.

For data storage this project will rely on PostgreSQL. This is a powerful, open source object-relational database which is well suited for the type of data we are working with.

For processing and analysis we will rely on Spark, which has a deep library for classification and analytics but also is very scalable as more resources can be added to a cluster very easily.

Thinking about how we can go to one small slice of data to the entire dataset over all available years ahead of time will save us a lot of headache going forward. To help us in this endeavor I will be using Docker Compose to run these services. This setup will be helpful when we scale up operations as we will see.

If you have not already, please follow the instructions in the [README.md](file:///../README.md).

In [14]:
# after the setup we are ready to load the data we have downloaded into our database
import load
load.load()

Loading started
Establising connection to database hmda_db listening on 0.0.0.0, port 54320 with user name: postgres.


OperationalError: could not connect to server: Cannot assign requested address (0x00002741/10049)
	Is the server running on host "0.0.0.0" and accepting
	TCP/IP connections on port 54320?


### Exploratory Data Analysis, Transformation, & Feature Engineering
Now that we have our data loaded, we can begin to do some initial analysis. As we step through the available data we should take care to think about how it can be interpreted and to make sure we set ourselves up for success by looking at abnormal features like outliers or highly skewed distributions.

We have done our job in getting the data together and stored. But now we need to make sure we have informative features. In my humble opinion, this is where a lot of value can be added or a things can go wrong so we will spend a fair amount of time in this section. Before we can even truly to any exploratory analysis we will need to preprocess a lot of this data so we can make better sense of it.

First and most importantly, if we take a look at the variable we will be trying to predict/classify we can it can take a few different forms.

In [ ]:
df = pd.DataFrame(set(data_lar['action_taken_name'])).rename(columns={0:'action_taken_name'})
df.style.set_properties(**{'width':'100%', 'text-align':'center'})

We need to define what meets our criteria. For this project we are interested in approvals. So if the loan is originated that is an approval, but so is if the application was approved by it was withdrawn by the applicant. On the other hand, we will define application denied by the financial instituion as unapproved but also if the preapproval request was denied by the instituion.

This leave several actions that we will filter out of our dataset. These will be:
1. When loans are puchased by other institutions
2. For incompleteness
3. When the applicant withdraws the application

In [ ]:
data_lar['action_taken'] = pd.to_numeric(data_lar['action_taken'])
lar = data_lar[(data_lar['action_taken'] != 6)]
lar = lar[(lar['action_taken'] != 5)]
lar = lar[(lar['action_taken'] != 4)]
"""dropped {} observations""".format(data_lar.shape[0] - lar.shape[0])

Another important filter we should impose is when income is missing from the dataset. Income is likely to be one of the most important features here, so if that isn't included in the data we shouldn't include it for now.

In [ ]:
start_obs = lar.shape[0]
lar = lar[(lar['applicant_income_000s'].apply(lambda x: math.isnan(float(x))) == False)]
"""dropped {} observations""".format(start_obs - lar.shape[0])

A brief note one dropping observations. Although beyond the scope of this project, it is often worth examining patterns within data where it is missing, where it is an outlier or otherwise strange. In this project we will be removing this data, but in practice just as having data can be informative where data is missing or especially is an outlier can be informative as well.

Moving on for now, we need to transform our data from text to our binary classification. 1 for approved and 0 for not approved.

It will be important to note how balanced/unbalanced our data is, since a highly unbalanced set will mean an model who selects the more likely outcome will appear to be more correct than it really may be.

In [ ]:
approved = np.zeros(lar.shape[0])
ids = (lar['action_taken'] == 1) | (lar['action_taken'] == 2)
approved[ids] = 1
lar['action_taken'] = approved

In [ ]:
lar[['action_taken','action_taken_name']].head()

In [ ]:
plt.figure(figsize=(8,6))
plt.hist(lar['action_taken'],facecolor='#3F4B8C',alpha=0.8)
plt.ylabel('Count')
plt.xlabel('Probability')
plt.title('Unbalance Approval/Non-Approval');

While our metric of interest is unbalanced, it is not extremely so.

I have already mentioned that I expect income to be an important feature of our data. However, it comes in a few different forms. Let's examine our income data and look at how we might transform them below.

In [ ]:
lar['applicant_income_000s'] = pd.to_numeric(lar['applicant_income_000s'])
plt.figure(figsize=(8,6))
plt.hist(lar['applicant_income_000s'],100,facecolor='#3F4B8C',alpha=0.8)
plt.ylabel('Count')
plt.xlabel('Income')
plt.title('Applicant Income (in thousands USD)');

We can see here that this data has a very log-normal looking distribution (i.e. very long right tail). Most models that we will consider have an underlying assumption that the data is normally distributed. It will be important for us to keep the underlying model assumptions in mind through our analysis process.

To address this in the income data, we will perform a log transformation. 

In [ ]:
plt.figure(figsize=(8,6))
plt.hist(lar['applicant_income_000s'].apply(lambda x: math.log(x,10)),100, \
        facecolor='#3F4B8C',alpha=0.8)
plt.ylabel('Count')
plt.xlabel('Income -- Log 10 Scale')
plt.title('Applicant Income (in thousands USD)');

First, you may notice that I choose a base 10 for the log transform. In this case I prefer to work on base 10 for interpretation as it tends to be easier to work in units of 10. Also notice that all of our values are above zero. We could scale this distribution to be above/beow average being above and below zero. This again comes down to distribution. I don't expect above or below average income to be especially important for acceptance, rather it will be proportional to the loan amount. I will always default to less transformation where possible with a keen focus on interpretation.

You will also notice now that the ditribution is much more normal looking, but we can also see a very strang blip around zero. Under closer inspection, these are entries of $1 for applicant income. For our purposes, we will filter these outliers from our dataset as they are likely an entry or other type of input error.

In [ ]:
start_obs = lar.shape[0]
lar = lar[(lar['applicant_income_000s'] != 1)]
"""dropped {} observations""".format(start_obs - lar.shape[0])

In [ ]:
"""The Range of our data is from {} up to {}""".format(min(lar['applicant_income_000s']), \
                                                      max(lar['applicant_income_000s']))

Looking at the new range of our data, we can still see an arbitrarily low number of 2 and an equally suspcious high number of 9999. While these are also likely to be entries which are less than accurate, just as in the case of missing data can have information I have kept these in the dataset under they assumption that they can also be informative. For instance, perhaps they represent sentiment of the loan officer. In any case, looking at our new distribution they don't cause the same type of outlier problem we witnessed before

In [ ]:
plt.figure(figsize=(8,6))
plt.hist(lar['applicant_income_000s'].apply(lambda x: math.log(x,10)),100, \
        facecolor='#3F4B8C',alpha=0.8)
plt.ylabel('Count')
plt.xlabel('Income -- Log 10 scale')
plt.title('Applicant Income (in thousands USD)');

Taking a look at another example variable, let's look at the number of units in an area that are built to house less than 5 families. This is a good metric, beyond population, for density of an area.

In [ ]:
lar['number_of_1_to_4_family_units'] = pd.to_numeric(lar['number_of_1_to_4_family_units'])
plt.figure(figsize=(8,6))
plt.hist(lar['number_of_1_to_4_family_units'], 100, facecolor='#3F4B8C',alpha=0.8)
plt.ylabel('Count')
plt.xlabel('Living Density')
plt.title('Number of Under 5 Family Homes');

Again here we have a very strong right tailed distribution. However, in this case moving to a log scale would only shift that right tail to a left tail. Furthermore, looking at the Shapiro-Wilk test of normaily the transformation would move the metric from 0.92 to 0.99. I would not consider that to be a large enough change to be compelling.

In this case, I do think scaling the data to be above or below average has more interpretive power. So this is the route I will take when scaling.

In [ ]:
from sklearn import preprocessing
plt.figure(figsize=(8,6))
plt.hist(preprocessing.scale(lar['number_of_1_to_4_family_units']), 100, facecolor='#3F4B8C',alpha=0.8)
plt.ylabel('Count')
plt.xlabel('Living Density -- Normalized')
plt.title('Number of Under 5 Family Homes');

In [ ]:
income_data = pd.DataFrame({'loan_amount': pd.to_numeric(lar['loan_amount_000s']), \
                            'income': pd.to_numeric(lar['applicant_income_000s'])})
jplot = sns.jointplot(y='loan_amount',x='income', data=income_data, kind='reg', height=10)